# 패키지 임포트

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import re
import json
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

# 데이터 로드

In [2]:
train_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-train-stemmed.csv')
valid_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-dev-stemmed.csv')
test_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-test-stemmed.csv')
train_df.head()

,text,edge,intent
0,champion product approv stock split champion p...,champion product approv stock split champion p...,earn
1,comput termin system cpml complet sale comput ...,comput termin system cpml complet sale comput ...,acq
2,cobanco inc cbco year net shr ct dlr net asset...,cobanco inc cbco year net shr ct dlr net asset...,earn
3,intern inc qtr jan oper shr loss two ct profit...,intern inc qtr jan oper shr loss two ct profit...,earn
4,brown forman inc bfd qtr net shr dlr ct net ml...,brown forman inc bfd qtr net shr dlr ct net ml...,earn


In [3]:
# train_df와 valid_df 결합
train_val_df = pd.concat([train_df, valid_df])

In [4]:
textcuboid=np.load('C:/Users/User/Desktop/r8/1-Channel textcuboid_r8(elmo).npy')
textcuboid_test=np.load('C:/Users/User/Desktop/r8/1-Channel textcuboid_test_r8(elmo).npy')

# 전처리

In [5]:
y_train=list(train_val_df['intent'])
y_test=list(test_df['intent'])

In [6]:
y=y_train+y_test

In [7]:
encoder=LabelEncoder()
encoder.fit(y)
label=encoder.transform(y)

y_train=list(label[:5484])
y_test=list(label[5484:])

In [8]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [9]:
textcuboid_train=textcuboid[:4937]
textcuboid_val=textcuboid[4937:]
y_train1=y_train[:4937]
y_val=y_train[4937:]

In [10]:
y_test1=y_test

# 모델 빌드


In [11]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, save_freq=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [12]:
#ELMo channel의 학습
ELMo_input = Input(shape=(237, 256))
conv1 = Conv1D(512, 1, padding='valid', activation='relu')(ELMo_input)
pooling = GlobalMaxPooling1D()(conv1)
pooling = Dropout(0.5)(pooling)
ELMo_x = Dense(256, activation='relu')(pooling)
ELMo_x = Dropout(0.5)(ELMo_x)
output_layer = Dense(8, activation='softmax')(ELMo_x)

model = Model(inputs=ELMo_input, outputs=output_layer)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 237, 256)]        0         
                                                                 
 conv1d (Conv1D)             (None, 237, 512)          131584    
                                                                 
 global_max_pooling1d (Globa  (None, 512)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                             

# 모델 학습

In [13]:
#모델 학습
history = model.fit(textcuboid_train, y_train1,callbacks=callbacks, epochs=50,batch_size=256,validation_data=(textcuboid_val, y_val))

Epoch 1/50
20/20 [==============================] - 1s 27ms/step - loss: 1.8228 - accuracy: 0.5850 - val_loss: 0.8227 - val_accuracy: 0.7130 - lr: 0.0010
Epoch 2/50
20/20 [==============================] - 0s 18ms/step - loss: 0.6660 - accuracy: 0.7800 - val_loss: 0.6298 - val_accuracy: 0.8154 - lr: 0.0010
Epoch 3/50
20/20 [==============================] - 0s 18ms/step - loss: 0.4768 - accuracy: 0.8416 - val_loss: 0.4503 - val_accuracy: 0.8702 - lr: 0.0010
Epoch 4/50
20/20 [==============================] - 0s 21ms/step - loss: 0.3640 - accuracy: 0.8829 - val_loss: 0.3337 - val_accuracy: 0.9013 - lr: 0.0010
Epoch 5/50
20/20 [==============================] - 0s 18ms/step - loss: 0.2946 - accuracy: 0.9048 - val_loss: 0.2772 - val_accuracy: 0.9232 - lr: 0.0010
Epoch 6/50
20/20 [==============================] - 0s 18ms/step - loss: 0.2376 - accuracy: 0.9216 - val_loss: 0.2452 - val_accuracy: 0.9250 - lr: 0.0010
Epoch 7/50
20/20 [==============================] - 0s 21ms/step - loss: 0.2

In [14]:
model.evaluate(textcuboid_test,y_test1)

69/69 [==============================] - 0s 2ms/step - loss: 0.0847 - accuracy: 0.9749


[0.08473405987024307, 0.9748743772506714]

In [21]:
model = load_model('./save model/best_model_1-Channel Tensor(ELMo).h5')
model.evaluate(textcuboid_test,y_test1)

69/69 [==============================] - 0s 2ms/step - loss: 0.0814 - accuracy: 0.9772


[0.08144426345825195, 0.9771585464477539]

In [22]:
y_pred = model.predict(textcuboid_test)
y_pred_labels = np.argmax(y_pred, axis=1)
macro_f1 = f1_score(y_test, y_pred_labels, average='macro')
print("Macro_F1 스코어:", macro_f1)
weighted_f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("weighted_F1 스코어:", weighted_f1)

69/69 [==============================] - 0s 2ms/step
Macro_F1 스코어: 0.9221368526387348
weighted_F1 스코어: 0.9770045106868248


In [ ]:
35에포크 drop1,2 활성화 512 97.7